<a href="https://colab.research.google.com/github/dave20874/RapidReact-Kicker/blob/main/Kicker-Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import math

This notebook develops some ideas for a pinball-style kicker to launch CARGO (oversized tennis balls) as part of 2022 FIRST Robotics Competition game, Rapid React.

The analysis relies on the concept of conservation of energy.  A motor will push back plunger (performing work), compressing a spring (storing potential energy).  When the plunger is released, the spring propels the plunger forward (converting to kinetic energy).  At the end of it's stroke, the plunger strikes the ball, transferring some kinetic energy to it.  The ball then flies up, it's energy transitioning smoothly to gravitational potential energy as it rises.

Let's begin by defining some values we'll be using:


In [68]:
# Gravitational acceleration, meters per second
# (For FRC matches held on Earth)
g = 9.8

# mass of the CARGO ball, kg
m_ball = 0.27

# radius of CARGO ball, meters
ball_radius = 0.12

# In a collision, the ball only retains about 80% of it's kinetic energy.
ball_loss = 0.80

# mass of the plunger, kg
m_plunger = 0.400

# pull distance of the plunger, meters
d_plunger = 0.06

# spring constant, Newtons per meter
# We will use 6 springs and measured 1.5 kg of force at 1cm (0.01m) compression
k_one_spring = 1.5*g / 0.01
num_springs = 6
k_spring = num_springs * k_one_spring

# Initial height of ball before shot, meters
h_shooter = 1.0

# Goal height, meters.  (8 feet 8 inches)
h_goal = 2.64

# Upper goal radius, meters
goal_radius = 0.61

# Launch angle, stored as radians.  (0 is horizontal, PI/2 is straight up)
# The function math.radians converts degrees into radians for readability.
theta_launch = math.radians(60)

Now we can define some useful functions.  These functions compute the amount of energy in various parts of our system and convert energy back into values like distances and speeds.

In [69]:
# Spring stored energy
# For a linear spring with spring constant, k, compressed by distance d.
def spring_energy(k, d):
  energy = 0.5 * k * d * d
  return energy

# Compute spring stretch amount for a spring with constant, k, and energy e
# since spring energy, e = 1/2 * k * d^2, we have d = sqrt(2*e/k)
def spring_stretch(k, e):
  d = math.sqrt(2.0*e/k)
  return d

# Get kinetic energy of a mass, m, moving at velocity, v
# e = 1/2 * m * v^2
def vel_to_ke(m, v):
  e = 0.5 * m * v * v
  return e

# Get velocity of a mass, m, with kinetic energy e
# since kinetic energy, e = 1/2*m*v^2, we have v = sqrt(2*e/m)
def ke_to_vel(e, m):
  v = math.sqrt(2.0 * e / m)
  return v

# Get potential energy of a mass, m, at height, h
def h_to_pe(m, h):
  e = m * g * h
  return e

# Get height of a mass, m, with potential energy e
def pe_to_h(e, m):
  h = e / (m * g)
  return h
  

Now we can ask some questions.

How much energy does the spring store, when it's compressed?  This is the amount of energy we're starting with.

In [70]:
# Starting energy
e = spring_energy(k_spring, d_plunger)
print(f"Starting energy: {e:4.3f}")


Starting energy: 15.876


And, quick question, how high would a CARGO go if all this energy were transferred to it?

In [71]:
# Height of a ball with this much energy added to it.
h = pe_to_h(e, m_ball)
print(f"How high could we shoot, above launcher? {h:4.3f} meters")
print(f"That is {h+h_shooter:4.3f} meters above floor")
print(f"And {h+h_shooter-h_goal:4.3f} meters above the high goal.")


How high could we shoot, above launcher? 6.000 meters
That is 7.000 meters above floor
And 4.360 meters above the high goal.


Using Energy let us compute the height of a launched ball directly from the compression of a spring.  Nice!  But we need to take a deeper look at the whole launch process.  What if not all that energy goes into the ball?  And what if we are launching the ball at an angle instead of straight up?

First, lets look at the collision of the plunger with the ball.  Does all the energy go to the ball?  No!  How much the ball receives depends on the mass of the plunger and the mass of the ball.

If there is no friction in the plunger mechanism, we can assume all the spring energy is transferred to the plunger just before it strikes the ball.  So, knowing the spring's energy and the plunger's mass, we can compute the speed of the plunger.

In [72]:
v_plunger = ke_to_vel(e, m_plunger)
print(f"Plunger is moving {v_plunger:4.3f} meters per second")

Plunger is moving 8.910 meters per second


Then we use an equation for computing the effect of a collision between two masses in one dimension.  

In this formula, m1 and m2 are the masses of two objects.  Before the collision they have velocities u1 and u2.  After, they have velocities v1 and v2.

We will evaluate the kinetic energy of the ball and plunger at this point to see how efficient our collision was.

In [73]:
# Compute post-collision velocities, v1 and v2, for objects with mass m1, m2 colliding with velocities u1, u2
def collide(m1, u1, m2, u2):
  v1 = (u1*(m1-m2)+2*m2*u2) / (m1+m2)
  v2 = (u2*(m2-m1)+2*m1*u1) / (m1+m2)
  return (v1, v2)

(v1, v2) = collide(m_plunger, v_plunger, m_ball, 0.0)
print(f"After an elastic collision, the ball would have velocity {v2:4.3f} meters per second")
print(f"The plunger would have velocity {v1:4.3f} meters per second.")
e_ball = vel_to_ke(m_ball, v2)
e_plunger = vel_to_ke(m_plunger, v1)
print(f"The ball has {e_ball:4.3f} Joules after collision.")
print(f"That is {100*e_ball/e:4.3f}% of the energy we started with.")

After an elastic collision, the ball would have velocity 10.638 meters per second
The plunger would have velocity 1.729 meters per second.
The ball has 15.278 Joules after collision.
That is 96.235% of the energy we started with.


TODO: Add analysis for optimal plunger mass.

We know the ball dissipates energy in collisions, though.  So we should reduce it's energy and re-evaluate how high it might go

In [74]:
e_ball = e_ball * ball_loss
print(f"Ball's adjusted energy: {e_ball:4.3f} Joules")
v_ball = ke_to_vel(e_ball, m_ball)
print(f"Ball's adjusted speed: {v_ball:4.3f} meters per second")
h_max = h_shooter + pe_to_h(e_ball, m_ball)
print(f"It can reach a max height of {h_max:4.3f} meters above the floor.")
print(f"That is {h_max-h_goal:4.3f} meters above the goal.")

Ball's adjusted energy: 12.223 Joules
Ball's adjusted speed: 9.515 meters per second
It can reach a max height of 5.619 meters above the floor.
That is 2.979 meters above the goal.


What sort of trajectories are possible with this much ball velocity?  We can answer this by looking first at the ball's velocity when it is at the opening of the high goal.  We can evaluate the potential energy at the goal high, subtract that from the total energy and the remaining part must be kinetic energy.  From there we get the velocity.

In [75]:
pe_goal = h_to_pe(m_ball, h_goal)
ke_at_goal = e_ball - pe_goal
v_at_goal = ke_to_vel(ke_at_goal, m_ball)
print(f"Velocity at goal entrance: {v_at_goal:4.3f} meters per second")


Velocity at goal entrance: 6.228 meters per second


Next, we can evaluate trajectories with that velocity, entering the goal at angles from vertical (90 degrees) to horizontal (0 degrees).  For each landing angle, we'll compute the max height above the goal, the horizontal distance from the goal when ascending, the launch distance from center of the field and the launch angle from the robot.  Some of these trajectories will be invalid because they don't clear the edge of the goal (max height less than ball radius horizontal ascent distance less than goal radius)

In [76]:
# Given the goal velocity and angle, compute trajectory data:
# returns (clearance_z, clearance_x, launch_d, launch_angle, is_valid)
def eval_trajectory(final_v, final_theta, launch_h):
  v_z = math.sin(final_theta)*final_v                # vertical speed at goal
  v_x = math.cos(final_theta)*final_v                # horizontal speed at goal
  t_apex_goal = v_z / g                         # time from apex to goal
  z_above_goal = 0.5 * g * t_apex_goal * t_apex_goal # height of apex above goal
  clearance_z = z_above_goal - ball_radius
  x_at_goal = 2.0 * t_apex_goal * v_x           # hor movement from h_goal to apex and back
  clearance_x = x_at_goal - goal_radius - ball_radius
  t_shot_apex = math.sqrt(2.0 * (h_goal+z_above_goal-h_shooter) / g)
  v_z_shooter = t_shot_apex * g
  launch_d = v_x * (t_shot_apex+t_apex_goal)
  launch_angle = math.atan(v_z_shooter / v_x)

  is_valid = True
  if clearance_z <= 0:
    is_valid = False
  if clearance_x <= 0:
    is_valid = False

  return (clearance_z, clearance_x, launch_d, launch_angle, is_valid)

print("   entry z-clear x-clear    dist   angle   valid")
for degrees in range(90, 28, -2):
  theta = math.radians(degrees)
  cz, cx, dist, angle, valid = eval_trajectory(v_at_goal, theta, h_shooter)
  print(f"{math.degrees(theta):8.3f}{cz:8.3f}{cx:8.3f}{dist:8.3f}{math.degrees(angle):8.3f}{valid:8}")

   entry z-clear x-clear    dist   angle   valid
  90.000   1.859  -0.730   0.000  90.000       0
  88.000   1.857  -0.454   0.325  88.521       0
  86.000   1.850  -0.179   0.648  87.043       0
  84.000   1.838   0.093   0.969  85.567       1
  82.000   1.821   0.361   1.287  84.093       1
  80.000   1.800   0.624   1.599  82.622       1
  78.000   1.774   0.880   1.905  81.156       1
  76.000   1.743   1.128   2.203  79.694       1
  74.000   1.709   1.368   2.493  78.239       1
  72.000   1.670   1.597   2.774  76.790       1
  70.000   1.628   1.815   3.044  75.349       1
  68.000   1.582   2.020   3.302  73.917       1
  66.000   1.532   2.212   3.547  72.495       1
  64.000   1.479   2.389   3.780  71.084       1
  62.000   1.423   2.552   3.998  69.685       1
  60.000   1.364   2.698   4.201  68.300       1
  58.000   1.303   2.828   4.389  66.928       1
  56.000   1.240   2.940   4.561  65.573       1
  54.000   1.175   3.035   4.716  64.236       1
  52.000   1.109   3